# Make main dataframe will all file matching info

This notebook is based on the previous one `analysis_notebooks/make_master_dataframe.ipynb` from May 2020 in the `pp_expt` repo. 

Here we update the notebook with:
1. new directory structure to data
2. the `executed_trials.xlsx` file, with new columns:
    - `LED_flash` 
    - Kavin's hand-checking comments and results for porpoise tracks and target positions (`KAVIN_TRACK_COMMENTS`, `TRACK_CHECKED`, `TARGETS_CHECKED`)
3. manual updates of `rough_chirp_time` in order to make chirp detection work. The cells are from notebook `20210117_update-master-df.ipynb`

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import linecache
import cv2

## Set up paths

In [2]:
raw_path = Path("../data_raw/")
analysis_path = Path("../data_processed/")

## Main info spreadsheet

Below `executed_trials.xlsx` was copied from the other repo in folder `pp_expt/trial_sequence/executed_trials.xlsx`. This file was created after the experiment was completed and more info was added to it as data analysis progressed.

In [3]:
trial_spreadsheet = analysis_path / 'all_info_csv/executed_trials.xlsx'
df_main = pd.read_excel(trial_spreadsheet)

In [4]:
df_main

,DATE,SESSION,PERSONNEL,TRIAL,LEFT,RIGHT,ANGLE,SPHEROID,ASPECT_RATIO,SEQUENCE,...,KAVIN_TRACK_COMMENTS,TRACK_CHECKED,TARGETS_CHECKED,LED_flash,TOUCH_FRAME,TRIAL_TO_INCLUDE,TRACK_ISSUES,LATE_TURN,EYECUP_ISSUE,TRACK_USED
0,20190625,1,"WJ, ML",0,T,X,1,X,0.0,1,...,NaN,NO,NO,N,8164.0,NaN,NaN,NaN,NaN,NaN
1,20190625,1,"WJ, ML",0,T,X,1,X,0.0,1,...,NaN,x,x,N,12094.0,NaN,NaN,NaN,NaN,NaN
2,20190625,1,"WJ, ML",1,T,C,1,L,2.9,1,...,NaN,x,x,N,15961.0,Y,NaN,NaN,NaN,NaN
3,20190625,1,"WJ, ML",2,C,T,1,L,2.9,1,...,NaN,x,x,N,20375.0,Y,NaN,NaN,NaN,NaN
4,20190625,1,"WJ, ML",3,T,C,1,L,2.9,1,...,NaN,x,x,N,23042.0,Y,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,20190704,3,ML,8,T,C,1,M,1.3,24,...,NaN,x,x,Y,23209.0,Y,NaN,NaN,NaN,NaN
244,20190704,3,ML,9,C,T,3,M,1.3,24,...,NaN,x,x,Y,24955.0,Y,NaN,NaN,NaN,NaN
245,20190704,3,ML,10,T,C,3,M,1.3,24,...,NaN,x,x,Y,26608.0,Y,NaN,NaN,NaN,NaN
246,20190704,3,ML,11,T,C,3,M,1.3,24,...,NaN,x,x,Y,28091.0,Y,NaN,NaN,NaN,NaN


## DTAG trial info

In [5]:
dtag_detect_path = analysis_path / 'dtag/dtag_detect_apr2020'

In [6]:
df_dtag_trial = pd.read_excel(
    dtag_detect_path / 'trial_time_diff_calc_sync_time.xlsx',
    skiprows=1, index_col=0
)

In [7]:
# Drop rows that are not real trial entries
df_dtag_trial.dropna(subset=['start_time', 'end_time'], inplace=True)
df_dtag_trial.reset_index(drop=True, inplace=True)

### Append corresponding wav file to DTAG trial info dataframe

In [8]:
all_wave_files = str(dtag_detect_path / 'all_dtag_wav_files.txt')
lines = linecache.getlines(all_wave_files)

In [9]:
wav_fname = []
for ln in range(3, len(lines)):
    wav_fname.append(lines[ln].rstrip())

In [10]:
pattern = re.compile('^20190(\d{3})/session(\d{1})/dtag/pp19_(\d{3})(\w{1})_\w+')

In [11]:
# Remove the 2nd wav file from the same trial
line_to_del = []
a_last = pattern.match(wav_fname[0])
for seq,wf in enumerate(wav_fname):
    if seq!=0:
        a_tmp = pattern.match(wf)
        if a_tmp.groups() == a_last.groups():
            line_to_del.append(seq)
        a_last = a_tmp
        
print('Remove the following entries since they are the second \(emtpy\) wav files:')
[wav_fname.pop(ln) for ln in line_to_del[::-1]] 

Remove the following entries since they are the second \(emtpy\) wav files:


['20190701/session2/dtag/pp19_182b_045.wav',
 '20190701/session1/dtag/pp19_182a_043.wav',
 '20190628/session3/dtag/pp19_179c_041.wav',
 '20190627/session2/dtag/pp19_178b_036.wav',
 '20190625/session3/dtag/pp19_176c_030.wav']

In [12]:
wlist = []
for seq, gp in enumerate(df_dtag_trial.groupby('session')):
    rep_num = gp[1]['session'].count()
    wlist += [wav_fname[seq]] * rep_num

In [13]:
df_dtag_trial['wav_file'] = wlist

## DLC labeled tracks

In [14]:
all_dlc_sessions = analysis_path / 'video_dlc_apr2020/combined_dlc_trial.csv'
all_dlc_sessions

PosixPath('../data_processed/video_dlc_apr2020/combined_dlc_trial.csv')

In [15]:
df_dlc = pd.read_csv(all_dlc_sessions, index_col=0)
df_dlc

,date,session,trial,gopro_video,labeled_h5
0,20190625,1,00_1,GOPR1466,GOPR1466_20190625_s1_t00_1DLC_resnet50_muri-fb...
1,20190625,1,00_2,GOPR1466,GOPR1466_20190625_s1_t00_2DLC_resnet50_muri-fb...
2,20190625,1,01,GOPR1466,GOPR1466_20190625_s1_t01DLC_resnet50_muri-fb-j...
3,20190625,1,02,GOPR1466,GOPR1466_20190625_s1_t02DLC_resnet50_muri-fb-j...
4,20190625,1,03,GOPR1466,GOPR1466_20190625_s1_t03DLC_resnet50_muri-fb-j...
...,...,...,...,...,...
243,20190704,3,08,GOPR1489,GOPR1489_20190704_s3_t08DLC_resnet50_muri-fb-j...
244,20190704,3,09,GOPR1489,GOPR1489_20190704_s3_t09DLC_resnet50_muri-fb-j...
245,20190704,3,10,GOPR1489,GOPR1489_20190704_s3_t10DLC_resnet50_muri-fb-j...
246,20190704,3,11,GOPR1489,GOPR1489_20190704_s3_t11DLC_resnet50_muri-fb-j...


In [16]:
df_dlc.groupby(['date','session']).count()

trial  gopro_video  labeled_h5
date     session                                
20190625 1            8            8           8
         2            8            8           8
         3           10           10          10
20190626 1           10           10          10
         2           10           10          10
         3           10           10          10
20190627 1           11           11          11
         2           11           11          11
         3           11           11          11
20190628 1           10           10          10
         2           12           12          12
         3           10           10          10
20190701 1           10           10          10
         2           11           11          11
         3           10           10          10
20190702 1           10           10          10
         2           12           12          12
         3           10           10          10
20190703 1           10           10          10
         2           11           11          11
         3           10           10          10
20190704 1           10           10          10
         2           11           11          11
         3           12           12          12

## Figure out any discrepancy between dataframes

In [17]:
df_cmp = pd.DataFrame(df_main.groupby(['DATE', 'SESSION'])['TRIAL'].count())
df_cmp['DTAG'] = np.array(df_dtag_trial.groupby('session')['filename'].count())
df_cmp['DLC'] = np.array(df_dlc.groupby(['date','session'])['trial'].count())
df_cmp.rename(columns={'TRIAL':'MAIN'}, inplace=True)

In [18]:
df_cmp

MAIN  DTAG  DLC
DATE     SESSION                 
20190625 1           8     8    8
         2           8     8    8
         3          10    10   10
20190626 1          10    10   10
         2          10    10   10
         3          10    10   10
20190627 1          11    11   11
         2          11    11   11
         3          11    11   11
20190628 1          10    10   10
         2          12    12   12
         3          10    10   10
20190701 1          10    10   10
         2          11    11   11
         3          10    10   10
20190702 1          10    10   10
         2          12    11   12
         3          10    10   10
20190703 1          10    10   10
         2          11    11   11
         3          10    10   10
20190704 1          10    10   10
         2          11    11   11
         3          12    12   12

In [19]:
df_cmp[
    (df_cmp['MAIN']!=df_cmp['DTAG'])
    | (df_cmp['MAIN']!=df_cmp['DLC'])
    | (df_cmp['DTAG']!=df_cmp['DLC'])
]

,,MAIN,DTAG,DLC
DATE,SESSION,,,
20190702,2,12,11,12


We can see that the trial number doesn't match for 20190702 session 2. This is because the main sheet contains trial #12 which did not have a chirp trigger (and therefore not included in `trial_time_diff_calc_sync_time` which is used to process DTAG recording.

## Add DTAG trial info and DLC label info to main dataframe

### Drop 20190702 session2 trial 12 on main dataframe and DLC record:

In [20]:
df_main.columns

Index(['DATE', 'SESSION', 'PERSONNEL', 'TRIAL', 'LEFT', 'RIGHT', 'ANGLE',
       'SPHEROID', 'ASPECT_RATIO', 'SEQUENCE', 'CHOICE', 'WJ_COMMENTS',
       'VIDEO_REVIEW_COMMENTS', 'VIDEO_TIME', 'VIDEO_NUM', 'OTHER_COMMENTS',
       'DARSHAN_COMMENTS', 'VIDEO_FILENAME', 'FRAME_NUM_START',
       'FRAME_NUM_END', 'KAVIN_TRACK_COMMENTS', 'TRACK_CHECKED',
       'TARGETS_CHECKED', 'LED_flash', 'TOUCH_FRAME', 'TRIAL_TO_INCLUDE',
       'TRACK_ISSUES', 'LATE_TURN', 'EYECUP_ISSUE', 'TRACK_USED'],
      dtype='object')

In [21]:
drop_indx = df_main[
    (df_main['DATE']==20190702) & (df_main['SESSION']==2) & (df_main['TRIAL']==12)
].index
drop_indx

Int64Index([173], dtype='int64')

In [22]:
df_dlc.drop(drop_indx, axis=0, inplace=True)
df_main.drop(drop_indx, axis=0, inplace=True)

### Append DTAG info to main dataframe

In [23]:
df_dtag_trial.rename(columns={'filename': 'hydro_filename', 
                              'trial_time': 'hydro_recording_time', 
                              'time_diff': 'hydro_time_diff', 
                              'sync_time': 'rough_chirp_time',
                              'start_time': 'dtag_trial_start_time', 
                              'end_time': 'dtag_trial_end_time', 
                              'chirp': 'see_chirp_in_dtag', 
                              'comments': 'dtag_trial_comments',
                              'wav_file': 'dtag_wav_file'}, 
                     inplace=True)

In [24]:
# discard 'session' since it is a duplication of what main dataframe already has
df_dtag_trial.drop(columns=['session'], inplace=True)

In [25]:
df_dtag_trial.reset_index(drop=True, inplace=True)
df_main.reset_index(drop=True, inplace=True)

In [26]:
df_main = df_main.join(df_dtag_trial)

In [27]:
df_main

,DATE,SESSION,PERSONNEL,TRIAL,LEFT,RIGHT,ANGLE,SPHEROID,ASPECT_RATIO,SEQUENCE,...,TRACK_USED,hydro_filename,hydro_recording_time,hydro_time_diff,rough_chirp_time,dtag_trial_start_time,dtag_trial_end_time,see_chirp_in_dtag,dtag_trial_comments,dtag_wav_file
0,20190625,1,"WJ, ML",0,T,X,1,X,0.0,1,...,NaN,40dBgain_trial__0_1.0V(peak)__25-06-2019_10_19...,2019-06-25 10:19:28,0 days 00:00:00.000000000,06:51.328000,6:49.100,7:00.200,Y,NaN,20190625/session1/dtag/pp19_176a_027.wav
1,20190625,1,"WJ, ML",0,T,X,1,X,0.0,1,...,NaN,40dBgain_trial__0_1.0V(peak)__25-06-2019_10_19...,2019-06-25 10:19:45,0 days 00:00:17.000000000,07:08.328000,7:07.500,7:29.500,Y,NaN,20190625/session1/dtag/pp19_176a_027.wav
2,20190625,1,"WJ, ML",1,T,C,1,L,2.9,1,...,NaN,40dBgain_trial__1_1.0V(peak)__25-06-2019_10_21...,2019-06-25 10:21:55,0 days 00:02:27.000000000,09:18.328000,9:16.900,9:39.900,Y,NaN,20190625/session1/dtag/pp19_176a_027.wav
3,20190625,1,"WJ, ML",2,C,T,1,L,2.9,1,...,NaN,40dBgain_trial__2_1.0V(peak)__25-06-2019_10_24...,2019-06-25 10:24:23,0 days 00:04:55.000000000,11:46.328000,11:44.800,12:06.800,Y,NaN,20190625/session1/dtag/pp19_176a_027.wav
4,20190625,1,"WJ, ML",3,T,C,1,L,2.9,1,...,NaN,40dBgain_trial__3_1.0V(peak)__25-06-2019_10_25...,2019-06-25 10:25:54,0 days 00:06:26.000000000,13:17.328000,13:15.200,13:36.100,Y,NaN,20190625/session1/dtag/pp19_176a_027.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,20190704,3,ML,8,T,C,1,M,1.3,24,...,NaN,trial_8_1.0V(peak)__04-07-2019_14_53_32.wav,2019-07-04 14:53:32,0 days 00:07:42.000000000,12:18.484000,12:17.830,12:40.950,Y,NaN,20190704/session3/dtag/pp19_185c_055.wav
243,20190704,3,ML,9,C,T,3,M,1.3,24,...,NaN,trial_9_1.0V(peak)__04-07-2019_14_54_32.wav,2019-07-04 14:54:32,0 days 00:08:42.000000000,13:18.484000,13:16.870,13:36.590,Y,NaN,20190704/session3/dtag/pp19_185c_055.wav
244,20190704,3,ML,10,T,C,3,M,1.3,24,...,NaN,trial_10_1.0V(peak)__04-07-2019_14_55_29.wav,2019-07-04 14:55:29,0 days 00:09:39.000000000,14:15.484000,14:10.480,14:31.940,Y,NaN,20190704/session3/dtag/pp19_185c_055.wav
245,20190704,3,ML,11,T,C,3,M,1.3,24,...,NaN,trial_11_1.0V(peak)__04-07-2019_14_56_14.wav,2019-07-04 14:56:14,0 days 00:10:24.000000000,15:00.484000,14:59.270,15:21.070,Y,NaN,20190704/session3/dtag/pp19_185c_055.wav


In [28]:
df_dlc.rename(columns={'trial': 'DLC_video_trial_label', 
                       'labeled_h5': 'DLC_labeled_h5'}, 
              inplace=True)

In [29]:
df_dlc.drop(columns=['date','session'], inplace=True)

In [30]:
df_dlc.reset_index(drop=True, inplace=True)

In [31]:
df_main = df_main.join(df_dlc)

In [32]:
df_main

,DATE,SESSION,PERSONNEL,TRIAL,LEFT,RIGHT,ANGLE,SPHEROID,ASPECT_RATIO,SEQUENCE,...,hydro_time_diff,rough_chirp_time,dtag_trial_start_time,dtag_trial_end_time,see_chirp_in_dtag,dtag_trial_comments,dtag_wav_file,DLC_video_trial_label,gopro_video,DLC_labeled_h5
0,20190625,1,"WJ, ML",0,T,X,1,X,0.0,1,...,0 days 00:00:00.000000000,06:51.328000,6:49.100,7:00.200,Y,NaN,20190625/session1/dtag/pp19_176a_027.wav,00_1,GOPR1466,GOPR1466_20190625_s1_t00_1DLC_resnet50_muri-fb...
1,20190625,1,"WJ, ML",0,T,X,1,X,0.0,1,...,0 days 00:00:17.000000000,07:08.328000,7:07.500,7:29.500,Y,NaN,20190625/session1/dtag/pp19_176a_027.wav,00_2,GOPR1466,GOPR1466_20190625_s1_t00_2DLC_resnet50_muri-fb...
2,20190625,1,"WJ, ML",1,T,C,1,L,2.9,1,...,0 days 00:02:27.000000000,09:18.328000,9:16.900,9:39.900,Y,NaN,20190625/session1/dtag/pp19_176a_027.wav,01,GOPR1466,GOPR1466_20190625_s1_t01DLC_resnet50_muri-fb-j...
3,20190625,1,"WJ, ML",2,C,T,1,L,2.9,1,...,0 days 00:04:55.000000000,11:46.328000,11:44.800,12:06.800,Y,NaN,20190625/session1/dtag/pp19_176a_027.wav,02,GOPR1466,GOPR1466_20190625_s1_t02DLC_resnet50_muri-fb-j...
4,20190625,1,"WJ, ML",3,T,C,1,L,2.9,1,...,0 days 00:06:26.000000000,13:17.328000,13:15.200,13:36.100,Y,NaN,20190625/session1/dtag/pp19_176a_027.wav,03,GOPR1466,GOPR1466_20190625_s1_t03DLC_resnet50_muri-fb-j...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,20190704,3,ML,8,T,C,1,M,1.3,24,...,0 days 00:07:42.000000000,12:18.484000,12:17.830,12:40.950,Y,NaN,20190704/session3/dtag/pp19_185c_055.wav,08,GOPR1489,GOPR1489_20190704_s3_t08DLC_resnet50_muri-fb-j...
243,20190704,3,ML,9,C,T,3,M,1.3,24,...,0 days 00:08:42.000000000,13:18.484000,13:16.870,13:36.590,Y,NaN,20190704/session3/dtag/pp19_185c_055.wav,09,GOPR1489,GOPR1489_20190704_s3_t09DLC_resnet50_muri-fb-j...
244,20190704,3,ML,10,T,C,3,M,1.3,24,...,0 days 00:09:39.000000000,14:15.484000,14:10.480,14:31.940,Y,NaN,20190704/session3/dtag/pp19_185c_055.wav,10,GOPR1489,GOPR1489_20190704_s3_t10DLC_resnet50_muri-fb-j...
245,20190704,3,ML,11,T,C,3,M,1.3,24,...,0 days 00:10:24.000000000,15:00.484000,14:59.270,15:21.070,Y,NaN,20190704/session3/dtag/pp19_185c_055.wav,11,GOPR1489,GOPR1489_20190704_s3_t11DLC_resnet50_muri-fb-j...


### Replace gopro_video column with full path

In [33]:
def replace_video_name(s):
    s['gopro_video'] = Path('%08d'%s['DATE']).joinpath(
        'session%s'%s['SESSION'], 'gopro_overhead', s['gopro_video']+'.MP4')
    return s

In [34]:
df_main = df_main.apply(replace_video_name, axis=1)

## Add tap sync info to main dataframe

In [35]:
video_dtag_sync_file = dtag_detect_path / 'video_dtag_sync_time_calc_dec2021.csv'

In [36]:
df_video_dtag_sync = pd.read_csv(video_dtag_sync_file, index_col=0)

In [37]:
df_video_dtag_sync.rename(columns={'vid_sync_num': 'tap_sync_video_num', 
                                   'dtag_read_start': 'tap_sync_dtag_start_time', 
                                   'dtag_read_end': 'tap_sync_dtag_end_time', 
                                   'vid_sync_frame': 'tap_sync_video_frame',
                                   'dtag_sync_comment': 'tap_sync_comments',
                                   'dtag2vid_sync_time': 'tap_sync_time_dtag2video'
                                  }, 
                          inplace=True)

In [38]:
df_video_dtag_sync.drop(columns=['Unnamed: 7', 'wav_file'], inplace=True)

In [39]:
df_main = df_main.merge(df_video_dtag_sync, 
                            how='outer', 
                            left_on=['DATE','SESSION'], 
                            right_on=['date','session'])

In [40]:
df_main.drop(columns=['date','session'], inplace=True)

## Correct `FRAME_NUM_START` and `TOUCH_FRAME` for trials in second and third videos

We need to correct for the frame number to account for the total frame number in the first video if the trial happened in the second video.

In [41]:
def correct_video_frame_num(s):
    if s.at['VIDEO_FILENAME'][:4]!='GOPR':
        v_seq_num = int(s.at['VIDEO_FILENAME'][3])
        v_id = s.at['VIDEO_FILENAME'][4:8]
        folder = Path(s.at['gopro_video']).parent

        dst_str = '%d_s%d_t%d' % (s.at['DATE'], s.at['SESSION'], s.at['TRIAL'])
        print(dst_str)
        print('--- trial in video #%d' % v_seq_num)
        
        f_num_ori = s.at['FRAME_NUM_START']

        if v_seq_num==1:  # add frame number from first video
            v0_name = 'GOPR'+v_id+'.MP4'
            cap = cv2.VideoCapture(str(Path(raw_path).joinpath(folder, v0_name)))
            v0_frame_num = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            s.at['FRAME_NUM_START'] += v0_frame_num
            s.at['TOUCH_FRAME'] += v0_frame_num
        elif v_seq_num==2:  # add frame number from first 2 videos
            v0_name = 'GOPR'+v_id+'.MP4'
            v1_name = 'GP01'+v_id+'.MP4'
            cap = cv2.VideoCapture(str(Path(raw_path).joinpath(folder, v0_name)))
            v0_frame_num = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            cap = cv2.VideoCapture(str(Path(raw_path).joinpath(folder, v1_name)))
            v1_frame_num = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            s.at['FRAME_NUM_START'] = s.at['FRAME_NUM_START'] + v0_frame_num + v1_frame_num
            s.at['TOUCH_FRAME'] = s.at['TOUCH_FRAME'] + v0_frame_num + v1_frame_num

        print('--- correcting FRAME_START_NUM from %d to %d' % 
              (f_num_ori, s.at['FRAME_NUM_START']))

    return s

In [42]:
df_main = df_main.apply(correct_video_frame_num, axis=1)

20190625_s1_t6
--- trial in video #1
--- correcting FRAME_START_NUM from 753 to 32583
20190625_s2_t8
--- trial in video #1
--- correcting FRAME_START_NUM from 857 to 32687
20190625_s3_t8
--- trial in video #1


--- correcting FRAME_START_NUM from 1380 to 33210
20190625_s3_t9
--- trial in video #1
--- correcting FRAME_START_NUM from 2862 to 34692
20190625_s3_t10
--- trial in video #1
--- correcting FRAME_START_NUM from 5488 to 37318
20190626_s2_t6
--- trial in video #1
--- correcting FRAME_START_NUM from 1520 to 33350
20190626_s2_t7
--- trial in video #1


--- correcting FRAME_START_NUM from 3476 to 35306
20190626_s2_t8
--- trial in video #1
--- correcting FRAME_START_NUM from 5283 to 37113
20190626_s2_t9
--- trial in video #1
--- correcting FRAME_START_NUM from 6960 to 38790
20190626_s2_t10
--- trial in video #1
--- correcting FRAME_START_NUM from 8741 to 40571
20190626_s3_t3
--- trial in video #1


--- correcting FRAME_START_NUM from 184 to 32014
20190626_s3_t4
--- trial in video #1
--- correcting FRAME_START_NUM from 2087 to 33917
20190626_s3_t5
--- trial in video #1
--- correcting FRAME_START_NUM from 3836 to 35666
20190626_s3_t6
--- trial in video #1
--- correcting FRAME_START_NUM from 6177 to 38007
20190626_s3_t7
--- trial in video #1


--- correcting FRAME_START_NUM from 7841 to 39671
20190626_s3_t8
--- trial in video #1
--- correcting FRAME_START_NUM from 9402 to 41232
20190626_s3_t9
--- trial in video #1
--- correcting FRAME_START_NUM from 11306 to 43136
20190626_s3_t10
--- trial in video #1
--- correcting FRAME_START_NUM from 13027 to 44857
20190627_s1_t9
--- trial in video #1


...

--- correcting FRAME_START_NUM from 10033 to 73693
20190701_s2_t9
--- trial in video #2
--- correcting FRAME_START_NUM from 11590 to 75250
20190701_s2_t10
--- trial in video #2
--- correcting FRAME_START_NUM from 13488 to 77148
20190701_s2_t11
--- trial in video #2


--- correcting FRAME_START_NUM from 14565 to 78225
20190702_s1_t10
--- trial in video #1
--- correcting FRAME_START_NUM from 191 to 32021
20190702_s2_t11
--- trial in video #1
--- correcting FRAME_START_NUM from 613 to 32443
20190702_s3_t10
--- trial in video #1
--- correcting FRAME_START_NUM from 642 to 32472
20190703_s2_t7
--- trial in video #1


--- correcting FRAME_START_NUM from 984 to 32814
20190703_s2_t8
--- trial in video #1
--- correcting FRAME_START_NUM from 2734 to 34564
20190703_s2_t9
--- trial in video #1
--- correcting FRAME_START_NUM from 4137 to 35967
20190703_s2_t10
--- trial in video #1
--- correcting FRAME_START_NUM from 5757 to 37587
20190703_s2_t11
--- trial in video #1


--- correcting FRAME_START_NUM from 7460 to 39290
20190704_s2_t8
--- trial in video #1
--- correcting FRAME_START_NUM from 479 to 32309
20190704_s2_t9
--- trial in video #1
--- correcting FRAME_START_NUM from 2692 to 34522
20190704_s2_t10
--- trial in video #1
--- correcting FRAME_START_NUM from 4321 to 36151
20190704_s2_t11
--- trial in video #1


--- correcting FRAME_START_NUM from 5694 to 37524


## Update `rough_chirp_time` in main dataframe

In [43]:
df_main.loc[(df_main['DATE']==20190628) 
              & (df_main['SESSION']==1) 
              & (df_main['TRIAL']==3), 'rough_chirp_time'] = '15:46.297000'

In [44]:
df_main.loc[(df_main['DATE']==20190628) 
              & (df_main['SESSION']==1) 
              & (df_main['TRIAL']==6), 'rough_chirp_time'] = '18:23.058000'

In [45]:
df_main.loc[(df_main['DATE']==20190628) 
              & (df_main['SESSION']==2) 
              & (df_main['TRIAL']==3), 'rough_chirp_time'] = '13:56.543000'

In [46]:
df_main.loc[(df_main['DATE']==20190628) 
              & (df_main['SESSION']==2) 
              & (df_main['TRIAL']==5), 'rough_chirp_time'] = '15:59.300000'

In [47]:
df_main.loc[(df_main['DATE']==20190628) 
              & (df_main['SESSION']==3) 
              & (df_main['TRIAL']==6), 'rough_chirp_time'] = '12:32.510000'

In [48]:
df_main.loc[(df_main['DATE']==20190628) 
              & (df_main['SESSION']==3) 
              & (df_main['TRIAL']==10), 'rough_chirp_time'] = '15:58.274000'

In [49]:
df_main.loc[(df_main['DATE']==20190701) 
              & (df_main['SESSION']==1) 
              & (df_main['TRIAL']==3), 'rough_chirp_time'] = '18:59.184000'

In [50]:
df_main.loc[(df_main['DATE']==20190701) 
              & (df_main['SESSION']==1) 
              & (df_main['TRIAL']==4), 'rough_chirp_time'] = '20:07.043000'

In [51]:
df_main.loc[(df_main['DATE']==20190701) 
              & (df_main['SESSION']==1) 
              & (df_main['TRIAL']==5), 'rough_chirp_time'] = '21:10.902000'

In [52]:
df_main.loc[(df_main['DATE']==20190701) 
              & (df_main['SESSION']==1) 
              & (df_main['TRIAL']==7), 'rough_chirp_time'] = '23:29.771000'

In [53]:
df_main.loc[(df_main['DATE']==20190701) 
              & (df_main['SESSION']==1) 
              & (df_main['TRIAL']==9), 'rough_chirp_time'] = '25:22.017000'

In [54]:
df_main.loc[(df_main['DATE']==20190702) 
              & (df_main['SESSION']==1) 
              & (df_main['TRIAL']==7), 'rough_chirp_time'] = '14:38.656000'

In [55]:
df_main.loc[(df_main['DATE']==20190703) 
              & (df_main['SESSION']==1) 
              & (df_main['TRIAL']==3), 'rough_chirp_time'] = '6:43.109000'

## Export the final main sheet!

In [56]:
df_main.to_csv(Path(analysis_path).joinpath('all_info_csv/main_info.csv'))